In [7]:
import torch

a = torch.tensor(2.0)
b = torch.tensor(3.0)
a.requires_grad = True
b.requires_grad = True
c = a + b
d = torch.tensor(10.0)
d.requires_grad = True
e = c * d
f = c / d
g = f ** 2

g.backward()
print(g.grad_fn)
print(f.grad_fn.next_functions)
print(a.grad)


((<AddBackward0 object at 0x1282f31c0>, 0), (<AccumulateGrad object at 0x128dfa5f0>, 0))
tensor(0.1000)


In [20]:
class Value:
    def __init__(self, data, requires_grad=False):
        self.data = data
        self.grad = 0
        self.requires_grad = requires_grad
        # stores previous value for refernce during backpropagation
        self.prev = []
        self._backward = lambda: None

    def __add__(self, other):
        # standard add operation
        result = Value(self.data + other.data)
        if self.requires_grad or other.required_grad:
            # automatically define requires_grad for the result value 
            result.requires_grad = True
            # save references for the result value
            result.prev = [self, other]
            # use to calculate gradient
            def _backward():
                if self.requires_grad:
                    print(result.grad * 1)
                    self.grad = result.grad * 1
                if other.requires_grad:
                    print(result.grad * 1)
                    other.grad = result.grad * 1
            # use _backward function to calculate gradients
            result._backward = _backward
        return result
    
    def __mul__(self, other):
        result = Value(self.data * other.data)
        if self.requires_grad or other.requires_grad:
            result.requires_grad = True
            result.prev = [self, other]
            def _backward():
                if self.requires_grad:
                    print(result.grad * other.data)
                    self.grad = result.grad * other.data
                if other.requires_grad:
                    print(result.grad * self.data)
                    other.grad = result.grad * self.data

            result._backward = _backward

        return result

    def backward(self):
        # Derivative of final input with respect to itself
        self.grad = 1.0
        topo = []
        visited = set()

        # transform graph into the liner ordering of its vertices (topological sort)
        def build_topo(value):
            if value not in visited:
                visited.add(value)
                for item in value.prev:
                    build_topo(item)
                topo.append(value)
        
        build_topo(self)
        # calculate gradients in a resersed order starting from the final output
        for i in reversed(topo):
            i._backward()

v1 = Value(torch.tensor(2.0), requires_grad = True)
v2 = Value(torch.tensor(3.0), requires_grad = False)
v3 = v1 + v2
v4 = Value(torch.tensor(10.0), requires_grad=True)
v5 = v4 * v3
v5.backward()

0
tensor(5.)
tensor(10.)
